# Demonstrate Classification

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from CONFIG import PREPROCESSED_DATA_PATH
from knn import KNN
from gensim.models import Word2Vec
import gensim.downloader
from classifier import Classifier
from word2vec import Word2VecModel
import math
import sklearn

In [2]:
# Set up some configs
SHOULD_RETRAIN_WORD2VEC = True
WORD2VEC_PATH = "custom_word_2_vec"

In [17]:
math.sqrt(70**2 + 60**2)

92.19544457292888

In [3]:
# Load data and shuffle
df_data = pd.read_parquet(PREPROCESSED_DATA_PATH)
df_data = df_data.sample(frac=1).reset_index(drop=True)

In [4]:
if SHOULD_RETRAIN_WORD2VEC:
    # Configure data
    dataset_size_limt = 100000 #len(df_data)#10000
    df_data = df_data[:dataset_size_limt]
    test_subset = df_data[:int(len(df_data)*0.2)]
    train_subset = df_data[int(len(df_data)*0.2):]
    
    # Fit the word2vec model
    training_tokens = np.array([x.split() for x in train_subset["title"]])
    w2v_custom = Word2VecModel()
    w2v_custom.fit(training_tokens, epochs=1024, batch_size=8192)
    
    # Save the word 2 vec model
    w2v_custom.save(WORD2VEC_PATH)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


Epoch 1/1024
9/9 - 1s - loss: 0.6926
Epoch 2/1024
9/9 - 1s - loss: 0.6915
Epoch 3/1024
9/9 - 1s - loss: 0.6899
Epoch 4/1024
9/9 - 1s - loss: 0.6878
Epoch 5/1024
9/9 - 1s - loss: 0.6850
Epoch 6/1024
9/9 - 1s - loss: 0.6812
Epoch 7/1024
9/9 - 1s - loss: 0.6767
Epoch 8/1024
9/9 - 1s - loss: 0.6716
Epoch 9/1024
9/9 - 1s - loss: 0.6655
Epoch 10/1024
9/9 - 1s - loss: 0.6581
Epoch 11/1024
9/9 - 1s - loss: 0.6514
Epoch 12/1024
9/9 - 1s - loss: 0.6437
Epoch 13/1024
9/9 - 1s - loss: 0.6352
Epoch 14/1024
9/9 - 1s - loss: 0.6267
Epoch 15/1024
9/9 - 1s - loss: 0.6201
Epoch 16/1024
9/9 - 1s - loss: 0.6109
Epoch 17/1024
9/9 - 1s - loss: 0.6026
Epoch 18/1024
9/9 - 1s - loss: 0.5955
Epoch 19/1024
9/9 - 1s - loss: 0.5874
Epoch 20/1024
9/9 - 1s - loss: 0.5792
Epoch 21/1024
9/9 - 1s - loss: 0.5737
Epoch 22/1024
9/9 - 1s - loss: 0.5672
Epoch 23/1024
9/9 - 1s - loss: 0.5604
Epoch 24/1024
9/9 - 1s - loss: 0.5549
Epoch 25/1024
9/9 - 1s - loss: 0.5518
Epoch 26/1024
9/9 - 1s - loss: 0.5427
Epoch 27/1024
9/9 - 1

Epoch 214/1024
9/9 - 1s - loss: 0.4326
Epoch 215/1024
9/9 - 1s - loss: 0.4315
Epoch 216/1024
9/9 - 1s - loss: 0.4300
Epoch 217/1024
9/9 - 1s - loss: 0.4352
Epoch 218/1024
9/9 - 1s - loss: 0.4329
Epoch 219/1024
9/9 - 1s - loss: 0.4289
Epoch 220/1024
9/9 - 1s - loss: 0.4296
Epoch 221/1024
9/9 - 1s - loss: 0.4328
Epoch 222/1024
9/9 - 1s - loss: 0.4302
Epoch 223/1024
9/9 - 1s - loss: 0.4319
Epoch 224/1024
9/9 - 1s - loss: 0.4324
Epoch 225/1024
9/9 - 1s - loss: 0.4336
Epoch 226/1024
9/9 - 1s - loss: 0.4300
Epoch 227/1024
9/9 - 1s - loss: 0.4324
Epoch 228/1024
9/9 - 1s - loss: 0.4314
Epoch 229/1024
9/9 - 1s - loss: 0.4270
Epoch 230/1024
9/9 - 1s - loss: 0.4303
Epoch 231/1024
9/9 - 1s - loss: 0.4332
Epoch 232/1024
9/9 - 1s - loss: 0.4274
Epoch 233/1024
9/9 - 1s - loss: 0.4260
Epoch 234/1024
9/9 - 1s - loss: 0.4279
Epoch 235/1024
9/9 - 1s - loss: 0.4269
Epoch 236/1024
9/9 - 1s - loss: 0.4295
Epoch 237/1024
9/9 - 1s - loss: 0.4271
Epoch 238/1024
9/9 - 1s - loss: 0.4229
Epoch 239/1024
9/9 - 1s -

9/9 - 1s - loss: 0.3848
Epoch 425/1024
9/9 - 1s - loss: 0.3838
Epoch 426/1024
9/9 - 1s - loss: 0.3862
Epoch 427/1024
9/9 - 1s - loss: 0.3857
Epoch 428/1024
9/9 - 1s - loss: 0.3829
Epoch 429/1024
9/9 - 1s - loss: 0.3814
Epoch 430/1024
9/9 - 1s - loss: 0.3860
Epoch 431/1024
9/9 - 1s - loss: 0.3860
Epoch 432/1024
9/9 - 1s - loss: 0.3831
Epoch 433/1024
9/9 - 1s - loss: 0.3803
Epoch 434/1024
9/9 - 1s - loss: 0.3796
Epoch 435/1024
9/9 - 1s - loss: 0.3820
Epoch 436/1024
9/9 - 1s - loss: 0.3821
Epoch 437/1024
9/9 - 1s - loss: 0.3838
Epoch 438/1024
9/9 - 1s - loss: 0.3855
Epoch 439/1024
9/9 - 1s - loss: 0.3828
Epoch 440/1024
9/9 - 1s - loss: 0.3801
Epoch 441/1024
9/9 - 1s - loss: 0.3824
Epoch 442/1024
9/9 - 1s - loss: 0.3802
Epoch 443/1024
9/9 - 1s - loss: 0.3789
Epoch 444/1024
9/9 - 1s - loss: 0.3839
Epoch 445/1024
9/9 - 1s - loss: 0.3803
Epoch 446/1024
9/9 - 1s - loss: 0.3824
Epoch 447/1024
9/9 - 1s - loss: 0.3772
Epoch 448/1024
9/9 - 1s - loss: 0.3834
Epoch 449/1024
9/9 - 1s - loss: 0.3816
E

Epoch 635/1024
9/9 - 1s - loss: 0.3562
Epoch 636/1024
9/9 - 1s - loss: 0.3574
Epoch 637/1024
9/9 - 1s - loss: 0.3559
Epoch 638/1024
9/9 - 1s - loss: 0.3590
Epoch 639/1024
9/9 - 1s - loss: 0.3575
Epoch 640/1024
9/9 - 1s - loss: 0.3549
Epoch 641/1024
9/9 - 1s - loss: 0.3511
Epoch 642/1024
9/9 - 1s - loss: 0.3581
Epoch 643/1024
9/9 - 1s - loss: 0.3556
Epoch 644/1024
9/9 - 1s - loss: 0.3590
Epoch 645/1024
9/9 - 1s - loss: 0.3592
Epoch 646/1024
9/9 - 1s - loss: 0.3584
Epoch 647/1024
9/9 - 1s - loss: 0.3554
Epoch 648/1024
9/9 - 1s - loss: 0.3565
Epoch 649/1024
9/9 - 1s - loss: 0.3574
Epoch 650/1024
9/9 - 1s - loss: 0.3563
Epoch 651/1024
9/9 - 1s - loss: 0.3556
Epoch 652/1024
9/9 - 1s - loss: 0.3556
Epoch 653/1024
9/9 - 1s - loss: 0.3543
Epoch 654/1024
9/9 - 1s - loss: 0.3520
Epoch 655/1024
9/9 - 1s - loss: 0.3530
Epoch 656/1024
9/9 - 1s - loss: 0.3559
Epoch 657/1024
9/9 - 1s - loss: 0.3551
Epoch 658/1024
9/9 - 1s - loss: 0.3571
Epoch 659/1024
9/9 - 1s - loss: 0.3569
Epoch 660/1024
9/9 - 1s -

9/9 - 1s - loss: 0.3476
Epoch 846/1024
9/9 - 1s - loss: 0.3482
Epoch 847/1024
9/9 - 1s - loss: 0.3425
Epoch 848/1024
9/9 - 1s - loss: 0.3452
Epoch 849/1024
9/9 - 1s - loss: 0.3505
Epoch 850/1024
9/9 - 1s - loss: 0.3442
Epoch 851/1024
9/9 - 1s - loss: 0.3481
Epoch 852/1024
9/9 - 1s - loss: 0.3461
Epoch 853/1024
9/9 - 1s - loss: 0.3473
Epoch 854/1024
9/9 - 1s - loss: 0.3417
Epoch 855/1024
9/9 - 1s - loss: 0.3474
Epoch 856/1024
9/9 - 1s - loss: 0.3486
Epoch 857/1024
9/9 - 1s - loss: 0.3493
Epoch 858/1024
9/9 - 1s - loss: 0.3487
Epoch 859/1024
9/9 - 1s - loss: 0.3506
Epoch 860/1024
9/9 - 1s - loss: 0.3486
Epoch 861/1024
9/9 - 1s - loss: 0.3429
Epoch 862/1024
9/9 - 1s - loss: 0.3478
Epoch 863/1024
9/9 - 1s - loss: 0.3411
Epoch 864/1024
9/9 - 1s - loss: 0.3432
Epoch 865/1024
9/9 - 1s - loss: 0.3474
Epoch 866/1024
9/9 - 1s - loss: 0.3461
Epoch 867/1024
9/9 - 1s - loss: 0.3456
Epoch 868/1024
9/9 - 1s - loss: 0.3503
Epoch 869/1024
9/9 - 1s - loss: 0.3447
Epoch 870/1024
9/9 - 1s - loss: 0.3481
E

# K-Fold Validation

In [5]:
c = Classifier(WORD2VEC_PATH)

In [6]:
df_data = df_data.sample(frac=1).reset_index(drop=True)
classes = np.array(df_data["categories"])
classes = classes[:10000]
titles = np.array(df_data["title"])
titles = titles[:10000]

In [7]:
kf = KFold(n_splits=10)
i = 0
for train_index, test_index in kf.split(titles):
    print("FOLD ", i)
    X_train, X_test = titles[train_index], titles[test_index]
    y_train, y_test = classes[train_index], classes[test_index]
    c.fit(X_train, y_train)
    
    predictions = c.predict(X_test)
    f_score = sklearn.metrics.f1_score(y_test, predictions, average='macro')
    precision = sklearn.metrics.precision_score(y_test, predictions, average='macro')
    recall = sklearn.metrics.recall_score(y_test, predictions, average='macro')
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    print("F Score:", f_score, "Precision", precision, "Recall", recall, "Accuracy", acc)

FOLD  0


/home/ruchtia/git/word2vec/word2vec/classifier.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings = np.array(word_embeddings)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ruchtia/git/word2vec/word2vec/classifier.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings 

F Score: 0.27450184964364355 Precision 0.34790050327571603 Recall 0.27403180081145845 Accuracy 0.583
FOLD  0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ruchtia/git/word2vec/word2vec/classifier.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings = np.array(word_embeddings)


F Score: 0.29567738013005757 Precision 0.36602474587699096 Recall 0.28797188758191894 Accuracy 0.557
FOLD  0


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ruchtia/git/word2vec/word2vec/classifier.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings = np.array(word_embeddings)


F Score: 0.3050888605049027 Precision 0.40337939475513135 Recall 0.30127047496506065 Accuracy 0.586
FOLD  0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ruchtia/git/word2vec/word2vec/classifier.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  word_embeddings = np.array(word_embeddings)


F Score: 0.27957341336615193 Precision 0.3340061710621563 Recall 0.2717972217523461 Accuracy 0.585
FOLD  0
F Score: 0.29957498379039815 Precision 0.3582452949231148 Recall 0.2993681106605503 Accuracy 0.612


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
